import pandas as pd

In [2]:
import pandas as pd

In [3]:
artisan = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\Artisan's Burgerbar.csv", error_bad_lines=False);
chinaspice = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\China Spice.csv', error_bad_lines=False);
eastasian = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\East Asian Restaurant.csv", error_bad_lines=False);
sawa = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Sawa Syrian Restaurant.csv', error_bad_lines=False);
venetianwalls = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Venetian Wall of Nicosia.csv', error_bad_lines=False);

tripadv_review = pd.concat([artisan, chinaspice, eastasian, sawa, venetianwalls])
tripadv_review

,r_points,r_text,place
0,40.0,Located in Nicosia. You can find make almost y...,Artisan's Burgerbar
1,20.0,Wrong cooking of the burger Salty salad medium...,Artisan's Burgerbar
2,40.0,I ordered the fried chicken sandwich. The chic...,Artisan's Burgerbar
3,30.0,not really that good. Quite expensive for what...,Artisan's Burgerbar
4,40.0,Nice place very good food but a bit pricy. Lov...,Artisan's Burgerbar
...,...,...,...
176,20.0,Nicosia is the only segregated city in the wor...,Venetian Wall of Nicosia
177,40.0,The Venetian Walls have been surrounding Nicos...,Venetian Wall of Nicosia
178,40.0,The Venetian walls surrounded Nicosia in a sta...,Venetian Wall of Nicosia
179,50.0,The first walls surrounding Nicosia were built...,Venetian Wall of Nicosia


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [9]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0      [located, nicosia, burger, serval, options, av...
1      [wrong, cooking, burger, salty, salad, medium,...
2      [ordered, fried, chicken, sandwich, chicken, c...
3      [good, expensive, eating, service, poor, consi...
4      [nice, place, good, food, pricy, lovely, envir...
                             ...                        
176    [nicosia, segregated, city, world, stellar, sh...
177    [venetian, walls, surrounding, nicosia, centur...
178    [venetian, walls, surrounded, nicosia, star, s...
179    [walls, surrounding, nicosia, built, century, ...
180    [venetian, walls, surround, city, lefkosa, tod...
Name: r_text, Length: 915, dtype: object

In [10]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 100,
                                           eval_every = None)

In [12]:
from pprint import pprint

In [13]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"food" + 0.019*"good" + 0.014*"service" + 0.009*"nice" + '
  '0.009*"dishes" + 0.008*"restaurant" + 0.008*"ordered" + 0.008*"tasty" + '
  '0.007*"excellent" + 0.006*"visited"'),
 (1,
  '0.028*"food" + 0.020*"service" + 0.016*"place" + 0.016*"restaurant" + '
  '0.011*"great" + 0.011*"quality" + 0.010*"chinese" + 0.010*"excellent" + '
  '0.010*"good" + 0.009*"staff"'),
 (2,
  '0.047*"burger" + 0.027*"burgers" + 0.015*"good" + 0.014*"food" + '
  '0.011*"cheese" + 0.011*"chicken" + 0.011*"like" + 0.009*"fries" + '
  '0.009*"beef" + 0.009*"quality"'),
 (3,
  '0.019*"time" + 0.019*"food" + 0.013*"place" + 0.012*"great" + '
  '0.011*"burgers" + 0.011*"burger" + 0.010*"service" + 0.009*"friendly" + '
  '0.008*"good" + 0.008*"staff"'),
 (4,
  '0.021*"walls" + 0.014*"nicosia" + 0.012*"city" + 0.012*"food" + '
  '0.011*"gate" + 0.010*"good" + 0.008*"famagusta" + 0.007*"burgers" + '
  '0.007*"restaurant" + 0.006*"time"'),
 (5,
  '0.065*"walls" + 0.033*"city" + 0.017*"nicosia" + 0.016

In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.039863  0.018580       1        1  28.110916
1      0.092214 -0.045554       2        1  16.941151
2      0.114179  0.105117       3        1  12.039364
4     -0.110661  0.030386       4        1   9.994153
3      0.032253  0.011848       5        1   7.749584
5     -0.213149  0.026698       6        1   6.606990
8      0.035538 -0.067615       7        1   6.267083
7      0.032997  0.061172       8        1   5.172060
0      0.025803 -0.085286       9        1   3.714609
6     -0.049038 -0.055346      10        1   3.404088, topic_info=          Term        Freq       Total Category  logprob  loglift
1461     walls  200.000000  200.000000  Default  30.0000  30.0000
2       burger  329.000000  329.000000  Default  29.0000  29.0000
205       city  147.000000  147.000000  Default  28.0000  28.0000
68        food  446.000000  446.000000  Default  27.0000  27.0000
20     nicosia  200.000000  200.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
338    waiters    2.604739   33.833290  Topic10  -5.7061   0.8161
3114  venetian    2.620636   47.654560  Topic10  -5.7000   0.4796
1828   chinese    2.803103   90.312462  Topic10  -5.6327  -0.0924
139       like    2.742005  105.465828  Topic10  -5.6547  -0.2695
32     service    2.674549  315.319458  Topic10  -5.6796  -1.3896

[787 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2229      7  0.641187           aalu
877       5  0.852138  accommodating
2659     10  0.691761    accompanies
1036      5  0.229535      according
1036      9  0.459069      according
...     ...       ...            ...
48        8  0.348923          wrong
48        9  0.116308          wrong
85        1  0.122182          yummy
85        2  0.122182          yummy
85        3  0.610910          yummy

[1718 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 3, 5, 4, 6, 9, 8, 1, 7])